## Load packages

In [2]:
using GPULMM, Flux, CUDA, SnpArrays, JLD, CSV, DataFrames, StatsBase, LinearAlgebra, ProgressMeter
CUDA.allowscalar(false)
ProgressMeter.ijulia_behavior(:clear);

## Load data

In [3]:
# 30217 samples, 13 covariates, 998187 SNPs
df = CSV.read("data/pheno/pheno.csv", DataFrame);

In [4]:
# Load covariates
X  = Matrix{Float32}(df[!,["Age_cont"]])
X  = (X .- mean(X,dims=1))./std(X,dims=1)

# Load and project traits
y  = Float32.(df[!,"Current_depression_cont"])
y  = (y .- mean(y))./std(y)
y.-= (X'X)\(X'y)

# Load kinship matrix
K  = JLD.load("data/kinship/kinship_projected.jld")["K"];

## Fit base variance components

In [5]:
# Put everything on the GPU
K_  = K |> cu
y_  = y |> cu;

In [6]:
# Fit base variance components
opt = Adam(0.01)
vc = fit_components(K_,y_;optimizer=opt,optimizer_iters=200);

Progress: 100%|███████████████████████████| Time: 0:00:56 ( 0.28  s/it)
  loss:        30703.843780402032
  components:  Float32[0.5600163, 0.7387179]


## Load SNP sets and SNP data

In [7]:
data = SnpData("data/plink/geno-ld08")
sets = open("data/sets/go.tsv") do f
   function p(l)
       recs = split(l,"\t")
       string(recs[1]) => string.(recs[2:end])
   end
   Dict(p(l) for l in eachline(f))
end
snp_cols = Dict(zip(data.snp_info.snpid,1:size(data.snparray,2)));

In [8]:
S = zeros(Float32,size(data.snparray)...)
Base.copyto!(S,data.snparray, model=ADDITIVE_MODEL, impute=true, center=true, scale=true);

## Test sets

In [9]:
st = SetTest(vc,y_);

In [10]:
progress = Progress(length(sets),showspeed=true)
pvals = Dict{String,Float64}()
for (set_id,set_snps) in sets
    cols = Int[]
    for s in set_snps
        r = get(snp_cols,s,0)
        if r>0
            push!(cols,r)
        end
    end
    g   = S[:,cols]
    g .-= (X'X)\(X'g)
    g_  = g |> cu
    
    _,pval = p_value(st,g_)
    pvals[set_id] = pval
    
    ProgressMeter.next!(progress; showvalues=[(:set_id,set_id),(:pval,pval),(:size,length(cols))])
end

Progress: 100%|███████████████████████████| Time: 0:00:37 ( 0.12  s/it)
  set_id:  GOBP_MATURE_RIBOSOME_ASSEMBLY
  pval:    0.7815254402371534
  size:    137
